In [19]:
import pandas as pd
import numpy as np

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [20]:
train = pd.read_csv('data/housing/train.csv', index_col='Id')
test = pd.read_csv('data/housing/test.csv', index_col='Id')

In [21]:
# Separate variables

X = train.drop('SalePrice', axis=1).copy()
y = train.SalePrice.copy()

In [22]:
# Separate columns
categorical_object_cols = [col for col in X.columns if X[col].dtype == "object"]

categorical_num_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64'] and 
                        X[col].nunique() <= 10]

numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64'] and 
                        X[col].nunique() > 10]


numerical_transformer = SimpleImputer(strategy="median")

categorical_obj_transformer = Pipeline(
    steps=[('imputer', SimpleImputer(strategy='most_frequent')),
           ('onehot', OneHotEncoder(handle_unknown="ignore"))
    ])

categorical_num_transformer = SimpleImputer(strategy="most_frequent")

preprocessor = ColumnTransformer(
    transformers=[
      ('num', numerical_transformer, numerical_cols),
      ('cat_obj', categorical_obj_transformer, categorical_object_cols),
      ('cat_num', categorical_num_transformer, categorical_num_cols)
])

model = XGBRegressor()

pipeline = Pipeline(
    steps=[
       ('preprocessor', preprocessor),
       ('model', model)
    ])

In [23]:
cv_scores = []
cv_preds = []

kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_indicies, valid_indicies) in enumerate(kf.split(X)):
    
    print("-"*15, "Fold", fold, "-"*15)
    X_train, X_valid = X.iloc[train_indicies], X.iloc[valid_indicies]
    y_train, y_valid = y.iloc[train_indicies].values, y.iloc[valid_indicies].values 

    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_valid)
    score = mean_squared_error(y_valid, y_pred, squared=False)
    print("Score: ", score)
    cv_scores.append(score)
    
    preds = pipeline.predict(test)
    cv_preds.append(preds)

--------------- Fold 0 ---------------
Score:  29280.72206071867
--------------- Fold 1 ---------------
Score:  29388.19158810715
--------------- Fold 2 ---------------
Score:  48541.47681453009
--------------- Fold 3 ---------------
Score:  28838.57023117395
--------------- Fold 4 ---------------
Score:  28037.578210406773


In [24]:
preds = np.mean(np.column_stack(cv_preds), axis=1)
